In [ ]:
import torch
import torch.nn as nn

import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification
from src.datamodules import Wav2Vec2DataModule, Wav2Vec2Dataset, AASISTDataset
from src.AASIST import AASISTModule


# 모델 및 특징 추출기 설정
model_name_or_path = "facebook/wav2vec2-base"

front_feature_extractor = AutoFeatureExtractor.from_pretrained(model_name_or_path)

feature_extractor = AutoModelForAudioClassification.from_pretrained(model_name_or_path)
bottleneck = nn.Linear(512, 23, bias=True)
classifier = AASISTModule()

In [ ]:
train_df = pd.read_csv("../dataset/train.csv")
test_df = pd.read_csv("../dataset/test.csv")
train_dset = Wav2Vec2Dataset(df=train_df, model_name=model_name_or_path, train_mode=True)
test_dset = Wav2Vec2Dataset(df=train_df, model_name=model_name_or_path, train_mode=False)
# train_dset = AASISTDataset(df=train_df, model_name=model_name_or_path, train_mode=True)
# test_dset = AASISTDataset(df=train_df, model_name=model_name_or_path, train_mode=False)

In [ ]:
train_audio, train_target = train_dset[4]
# train_audio = train_dset[7]
# train_audio = np.array(train_audio)

output = feature_extractor.base_model(train_audio.unsqueeze(0))

# feature = output.extract_features
# bs = feature.shape[0]
# feature = feature.reshape(bs, -1)

last_hidden, output = classifier(torch.rand(4, 80000))

In [ ]:
output.extract_features.shape

In [ ]:
res, _ = classifier(torch.rand(1, 768))

In [ ]:
print(train_target)
plt.imshow(output.extract_features[0].detach().cpu().numpy())

In [ ]:
plt.imshow(output.last_hidden_state[0].detach().cpu().numpy())

In [ ]:
test_audio = test_dset[4]
test_audio = np.array(test_audio)

output = model.base_model(torch.tensor(test_audio).unsqueeze(0))
output.extract_features.shape, output.last_hidden_state.shape

In [ ]:
plt.imshow(output.extract_features[0].detach().cpu().numpy())

In [ ]:
plt.imshow(output.last_hidden_state[0].detach().cpu().numpy())

In [ ]:
plt.plot(train_audio)

In [ ]:
plt.plot(test_audio)

In [ ]:
ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")

In [ ]:
ds[2]['audio']['array'].shape

In [ ]:
y, sr = librosa.load("/home/work/joono/joono/dataset/test/TEST_00000.ogg", sr=16000)

In [ ]:
y.shape

In [ ]:
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")
model = AutoModelForAudioClassification.from_pretrained("facebook/wav2vec2-base", num_labels=2)

In [ ]:
model.classifier

In [ ]:
model.projector

In [ ]:
model.base_model.encoder.layers[0].attention.k_proj

In [ ]:
list(model.children())[1]

In [ ]:
model.projector

In [ ]:
model.classifier

In [ ]:
model.base_model

In [ ]:
model.base_model.feature_extractor

In [ ]:
model.base_model.feature_projection

In [ ]:
model.base_model.encoder

In [5]:
import matplotlib.pyplot as plt
import torch
import librosa
from transformers import Wav2Vec2Model
from src.RawNet2 import RawNet2

feature_extractor = Wav2Vec2Model.from_pretrained(
            "facebook/wav2vec2-large-960h-lv60-self", 
            torch_dtype=torch.float32, 
            attn_implementation="sdpa", 
            num_hidden_layers=12,
        )
classifier = RawNet2()

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
sample_audio, _ = librosa.load("/home/joono/joono/dataset/test/TEST_00000.ogg", sr=16000)
# sample_audio = torch.tensor(sample_audio)
sample_audio = torch.rand(16000)

output1 = feature_extractor(sample_audio.unsqueeze(0))
feature = output1['extract_features']

output = classifier(feature)

print(output)

x1: torch.Size([1, 49, 512])
x2: torch.Size([1, 512, 49])
x3: torch.Size([1, 512, 24])
x4: torch.Size([1, 512, 12])
x5: torch.Size([1, 1024, 6])
x6: torch.Size([1, 1024, 3])
tensor([[ 0.2771,  0.3826, -0.3024, -0.3515]], grad_fn=<AddmmBackward0>)


In [ ]:
model = RawNet2()

In [ ]:
input = torch.rand(4, 59049)

input = torch.rand(4, 128, 20000)

output = model(input)

In [6]:
feature_extractor

Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
      (1-4): 4 x Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
      (5-6): 2 x Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=1024, bias=True)
    (dropout)